In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [2]:
# download data
train_data = datasets.FashionMNIST(
  root="data",
  train=True,
  download=True,
  transform=ToTensor()
)

test_data = datasets.FashionMNIST(
  root="data",
  train=False,
  download=True,
  transform=ToTensor()
)

In [3]:
batch_size = 64

train_dataloader = DataLoader(train_data, batch_size=batch_size) # convert into iterable of batches
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print(X.shape)
  print(y.shape, y.dtype)
  break

torch.Size([64, 1, 28, 28])
torch.Size([64]) torch.int64


In [4]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
# optimizing
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [6]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # Compute prediciton error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [7]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.304444 [    0/60000]
loss: 2.298505 [ 6400/60000]
loss: 2.285719 [12800/60000]
loss: 2.277902 [19200/60000]
loss: 2.260002 [25600/60000]
loss: 2.238669 [32000/60000]
loss: 2.239097 [38400/60000]
loss: 2.210358 [44800/60000]
loss: 2.208947 [51200/60000]
loss: 2.187405 [57600/60000]
Test Error: 
 Accuracy: 48.1%, Avg loss: 2.180581 

Epoch 2
-------------------------------
loss: 2.181034 [    0/60000]
loss: 2.181141 [ 6400/60000]
loss: 2.139420 [12800/60000]
loss: 2.152759 [19200/60000]
loss: 2.097204 [25600/60000]
loss: 2.053293 [32000/60000]
loss: 2.075538 [38400/60000]
loss: 2.006816 [44800/60000]
loss: 2.010810 [51200/60000]
loss: 1.955721 [57600/60000]
Test Error: 
 Accuracy: 55.1%, Avg loss: 1.946476 

Epoch 3
-------------------------------
loss: 1.965642 [    0/60000]
loss: 1.948004 [ 6400/60000]
loss: 1.852030 [12800/60000]
loss: 1.888397 [19200/60000]
loss: 1.759469 [25600/60000]
loss: 1.728680 [32000/60000]
loss: 1.748620 [38400/

In [9]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [10]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [11]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


In [12]:
x.shape

torch.Size([1, 28, 28])

In [13]:
y

9

In [14]:
pred # getting the logits

tensor([[-2.2371, -2.5178, -0.7874, -1.9172, -0.6785,  2.3149, -0.9650,  2.6023,
          1.7460,  2.9474]])

In [15]:
random_data = torch.rand(3, 28, 28) # creating a random set
# two ways to get the same value
print(random_data.shape)
print(random_data.size())

torch.Size([3, 28, 28])
torch.Size([3, 28, 28])


In [16]:
flatten = nn.Flatten() # instance to flatten the array
flat_image = flatten(random_data)
print(flat_image.size()) # note that the first dimension remains

torch.Size([3, 784])


In [17]:
layer1 = nn.Linear(in_features=28*28, out_features=20) # input is image flatten and output is 20 neurons
hidden1 = layer1(flat_image) # forward pass through the layer
print(hidden1.size())

torch.Size([3, 20])


In [18]:
print(f"Before ReLU: {hidden1}\n\n") # result of each activation per image
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}") # result after activation per image

Before ReLU: tensor([[ 3.9256e-01, -6.1266e-01, -4.4406e-01,  1.6300e-01,  1.2100e-01,
         -4.8544e-01,  8.6252e-02,  1.3206e-01,  5.0336e-02,  3.0496e-01,
          3.7245e-01, -3.4766e-01, -4.3637e-01, -3.7825e-01, -2.3388e-01,
         -7.5048e-01,  1.1577e-01, -6.4414e-04,  8.0040e-01, -2.0533e-02],
        [-9.3234e-02, -7.1480e-01, -3.9238e-01,  2.1280e-01,  2.6324e-01,
         -5.1239e-01,  2.2324e-01,  3.9740e-01,  2.2475e-01,  5.7993e-02,
          8.5548e-01, -2.9849e-01, -4.3808e-01, -5.6405e-01, -1.9851e-01,
         -1.6204e-01,  2.7664e-01, -1.0087e-01,  8.5103e-01, -4.0795e-02],
        [ 1.9529e-02, -3.1077e-01, -1.6430e-01,  5.4918e-02,  3.0292e-01,
         -4.1398e-01, -1.5894e-01,  3.0210e-01, -1.9161e-01,  2.2263e-01,
          6.7973e-01, -6.5344e-01, -6.9539e-02, -2.4517e-01, -7.9543e-02,
         -2.5453e-01,  1.6931e-01, -1.4008e-01,  7.3027e-01, -3.5497e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.3926, 0.0000, 0.0000, 0.1630, 0.1210,

In [19]:
# grab the parameters of the hidden layer
params = {}
for b in layer1.named_parameters():
  params[b[0]] = b[1]
params['weight']

Parameter containing:
tensor([[ 0.0340, -0.0028,  0.0182,  ..., -0.0025, -0.0026,  0.0179],
        [-0.0273,  0.0049,  0.0302,  ...,  0.0157, -0.0182, -0.0268],
        [-0.0199,  0.0338, -0.0342,  ..., -0.0131,  0.0027, -0.0324],
        ...,
        [-0.0353, -0.0105,  0.0063,  ...,  0.0185, -0.0183,  0.0186],
        [ 0.0279, -0.0056, -0.0181,  ...,  0.0082,  0.0150,  0.0341],
        [-0.0029,  0.0338, -0.0252,  ..., -0.0032, -0.0103,  0.0276]],
       requires_grad=True)